#### Core 2 — Degradation Prediction

목표:
상태 열화(degradation)는 최소 수준에서 예측 가능한가를 확인한다.

본 Core의 목적은
- 고성능 예측 ❌
- 복잡한 모델 ❌

오직:
상태 변화율이 완전히 무작위가 아님을 확인하는 것이다.

In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

df = pd.read_csv("../data/raw/health_timeseries_core_state.csv")
df["date"] = pd.to_datetime(df["date"])
df = (
    df.rename(columns={
        "user_id": "asset_id",
        "health_state_index": "state_value"
    })
    .sort_values(["asset_id", "date"])
    .reset_index(drop=True)
)

df.head()

,asset_id,date,mean_hr,hr_std,steps,calories,sleep_minutes,health_state_level,health_state_speed,state_value
0,1503960366,2016-04-12,NaN,NaN,13162,1985,327.0,-0.002802,0.000000,-0.002802
1,1503960366,2016-04-13,NaN,NaN,10735,1797,384.0,0.054147,0.056949,0.452790
2,1503960366,2016-04-14,NaN,NaN,10460,1776,NaN,NaN,NaN,NaN
3,1503960366,2016-04-15,NaN,NaN,9762,1745,412.0,0.064168,NaN,NaN
4,1503960366,2016-04-16,NaN,NaN,12669,1863,340.0,0.005229,NaN,NaN


In [12]:
df["delta_state"] = (
    df
    .groupby("asset_id")["state_value"]
    .diff()
)

df[["asset_id", "date", "state_value", "delta_state"]].head(10) # supervised delta 생성

,asset_id,date,state_value,delta_state
0,1503960366,2016-04-12,-0.002802,NaN
1,1503960366,2016-04-13,0.452790,0.455592
2,1503960366,2016-04-14,NaN,NaN
3,1503960366,2016-04-15,NaN,NaN
4,1503960366,2016-04-16,NaN,NaN
5,1503960366,2016-04-17,NaN,NaN
6,1503960366,2016-04-18,NaN,NaN
7,1503960366,2016-04-19,NaN,NaN
8,1503960366,2016-04-20,NaN,NaN
9,1503960366,2016-04-21,NaN,NaN


학습용 데이터 구성

In [13]:
# lag feature (이전 상태)
df["state_lag1"] = (
    df
    .groupby("asset_id")["state_value"]
    .shift(1)
)

train_df = df.dropna(subset=["delta_state", "state_lag1"]).reset_index(drop=True)

X = train_df[["state_lag1"]]
y = train_df["delta_state"]

X.shape, y.shape

((446, 1), (446,))

최소 모델 (Linear Regression)

In [14]:
model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

train_df["predicted_delta"] = y_pred
train_df["abs_error"] = np.abs(train_df["delta_state"] - train_df["predicted_delta"])

mae = mean_absolute_error(y, y_pred)
mae

0.524848776175937

예측 결과 저장

In [15]:
df["degradation_rate"] = (
    df
    .groupby("asset_id")["delta_state"]
    .rolling(window=7, min_periods=3)
    .mean()
    .reset_index(level=0, drop=True)
)

df[
    ["asset_id", "date", "state_value", "degradation_rate"]
].head(10)

,asset_id,date,state_value,degradation_rate
0,1503960366,2016-04-12,-0.002802,NaN
1,1503960366,2016-04-13,0.452790,NaN
2,1503960366,2016-04-14,NaN,NaN
3,1503960366,2016-04-15,NaN,NaN
4,1503960366,2016-04-16,NaN,NaN
5,1503960366,2016-04-17,NaN,NaN
6,1503960366,2016-04-18,NaN,NaN
7,1503960366,2016-04-19,NaN,NaN
8,1503960366,2016-04-20,NaN,NaN
9,1503960366,2016-04-21,NaN,NaN


prediction_results df (선택 산출물)

In [16]:
prediction_results = train_df[
    [
        "asset_id",
        "date",
        "delta_state",
        "predicted_delta",
        "abs_error"
    ]
].copy()

In [17]:
df.to_csv("../data/derived/core2_with_degradation_rate.csv", index=False)

print("✅ prediction_results.csv 저장 완료") # CSV 저장

✅ prediction_results.csv 저장 완료


## Core 2 결론

- 상태 변화율(delta)은 완전히 무작위가 아니다.
- 단순한 선형 모델로도 일정 수준의 예측 가능성이 존재한다.
- 그러나 이 예측값은 이후 의사결정에 **단독 의사결정 입력으로는 한계가 있다**.

이 한계는 Core 5 이후 단계에서 구조적으로 검증된다.